In [ ]:
import os
from typing import List
from dotenv import load_dotenv
from datahub.emitter.mcp import MetadataChangeProposalWrapper
from datahub.emitter.rest_emitter import DatahubRestEmitter
from datahub.metadata.schema_classes import (
    ChangeTypeClass,
    OwnershipClass,
    DomainsClass,
)

# Load environment variables
load_dotenv()

# Constants
DATAHUB_SERVER_URL = os.getenv('DATAHUB_SERVER_URL')
CATALOGUE_TOKEN = os.getenv('CATALOGUE_TOKEN')
TARGET_DOMAIN_URN = "urn:li:domain:XXXXXXXX"
PLATFORM = "XXXXXX"

def get_dataset_urns() -> List[str]:
    """
    Return a list of dataset URNs to update.
    You might need to modify this based on how you want to identify the datasets.
    """
    return [
"urn:li:dataset:(urn:li:dataPlatform:XXX,0XXXX,PROD)",
"urn:li:dataset:(urn:li:dataPlatform:XXX,0XXXX,PROD)"   ]

def create_domain_update_mcp(dataset_urn: str) -> MetadataChangeProposalWrapper:
    """Create MCP to update the dataset's domain."""
    return MetadataChangeProposalWrapper(
        entityType="dataset",
        entityUrn=dataset_urn,
        aspectName="domains",  # Correct aspect name
        aspect=DomainsClass(domains=[TARGET_DOMAIN_URN]),
        changeType=ChangeTypeClass.UPSERT
    )

def create_remove_owners_mcp(dataset_urn: str) -> MetadataChangeProposalWrapper:
    """Create MCP to remove all owners."""
    return MetadataChangeProposalWrapper(
        entityType="dataset",
        entityUrn=dataset_urn,
        aspectName="ownership",
        aspect=OwnershipClass(owners=[]),
        changeType=ChangeTypeClass.UPSERT
    )

def emit_to_datahub(emitter: DatahubRestEmitter, mcp: MetadataChangeProposalWrapper) -> None:
    """Emit MCP to DataHub with error handling."""
    try:
        emitter.emit_mcp(mcp)
        print(f"Successfully emitted MCP for: {mcp.entityUrn}")
    except Exception as e:
        print(f"Failed to emit MCP for: {mcp.entityUrn}")
        print(f"Error: {str(e)}")

def main():
    try:
        # Initialize emitter
        emitter = DatahubRestEmitter(
            gms_server=DATAHUB_SERVER_URL,
            token=CATALOGUE_TOKEN
        )

        # Get list of dataset URNs to update
        dataset_urns = get_dataset_urns()

        # Process each dataset
        for dataset_urn in dataset_urns:
            print(f"\nProcessing dataset: {dataset_urn}")

            # Update domain
            domain_mcp = create_domain_update_mcp(dataset_urn)
            emit_to_datahub(emitter, domain_mcp)

            # Remove owners
            owners_mcp = create_remove_owners_mcp(dataset_urn)
            emit_to_datahub(emitter, owners_mcp)

    except Exception as e:
        print(f"Unexpected error: {str(e)}")

if __name__ == "__main__":
    main()